# Capstone Project Title
___
**Author**: Evan Holder

## Overview
___

## Business Problem
___

### Import Libraries and Functions
___
Data manipulation, cleaning, massaging: pandas, numpy,
Modeling: sklearn, keras
Plotting: matplotlib
Custom functions: function.py

In [2]:
# Libraries for data cleaning, massaging:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import datetime as dt


# Modeling Libraries
from xgboost import XGBRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.feature_selection import RFE
from sklearn.linear_model import Lasso

import tensorflow as tf
from tensorflow import keras
from keras import layers, models, regularizers
from tensorflow.keras.layers import TimeDistributed
from sklearn.preprocessing import MinMaxScaler


# Plotting
import matplotlib.pyplot as plt
import seaborn as sns

# Maneuvering directories 
import os



# Import custom functions
os.chdir('../scripts')
from functions import split_data, sMAPE, SMAPE, compute_metrics, r2, impute_immediate_mean
os.chdir('../notebooks')

#from tensorflow.keras.preprocessing import timeseries_dataset_from_array
#from sklearn.model_selection import train_test_split, GridSearchCV
#from sklearn.metrics import r2_score


## Data
___

In [3]:
# Read in data
df_lag = pd.read_csv('../data/clean/df_clean_lag.csv', index_col=0, parse_dates=True)

## Methods
___

## Results
___

## Conclusion
___

## Next Steps
___